In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import httpx
from bs4 import BeautifulSoup
import re
import time

In [3]:
def get_carCode_from_listing(listing_id):
    try:
        r = httpx.get("https://www.sgcarmart.com/used_cars/info.php?ID="+str(listing_id))
        content = BeautifulSoup(r.content, 'html.parser')
        parent = content.find(class_="twoRow_info")
        link = parent.parent.find('a')
        carCode = re.search("CarCode=(.+?)\"",str(link)).group(1).replace("'","")
    except Exception as e:
        carCode = "MISSING"
        #print(carCode)
    return carCode

In [28]:
dic_f ={}
def get_fuel_type_on_carCode(carCode):
    if carCode == "MISSING":
        return "MISSING CAR CODE"
    try:
        if carCode in dic_f:
            return dic_f[carCode]
        query = "https://www.sgcarmart.com/new_cars/newcars_specs.php?CarCode="+str(carCode)
        r = httpx.get(query)
        content = BeautifulSoup(r.content, 'html.parser')
        fuel_type = content.find('td', text='Fuel type').nextSibling.nextSibling.text
        dic_f[carCode] = fuel_type
    except Exception as e:
        fuel_type = "ERROR ON SCRAPING PAGE"
    
    return fuel_type

In [33]:
def get_fuel_value(x):
    if 'petrol-electric' in str.lower(x):
        return "petrol-electric"
    if 'diesel' in str.lower(x):
        return 'diesel'
    if 'electric' in str.lower(x):
        return 'electric'
    if 'petrol' in str.lower(x):
        return 'petrol'
    return ''

In [30]:
df_train = pd.read_csv("./train.csv")
df_test = pd.read_csv("./test.csv")
df_f = pd.concat([df_train, df_test])[["listing_id","fuel_type"]].reset_index().drop(columns="index")
a = time.time()
df_f["CarCode"] = df_f.listing_id.apply(get_carCode_from_listing)
print(time.time()-a)

7616.4819185733795


In [31]:
df_f.to_csv("data_listing_carcode")

In [40]:
df_f["extracted_fuel_type"] = df_f.CarCode.apply(get_fuel_type_on_carCode)

In [43]:
df_f["extracted_fuel_type_standardized"] = df_f.extracted_fuel_type.apply(get_fuel_value)

In [42]:
df_f.extracted_fuel_type.value_counts()

Petrol              14511
MISSING CAR CODE     4720
Diesel (Euro VI)      956
Petrol-Electric       921
Diesel (Euro V)       590
Diesel (Euro IV)       60
Electric               26
Name: extracted_fuel_type, dtype: int64

In [44]:
df_f.extracted_fuel_type_standardized.value_counts()

petrol             14511
                    4720
diesel              1606
petrol-electric      921
electric              26
Name: extracted_fuel_type_standardized, dtype: int64

In [47]:
df_f.fuel_type.value_counts(dropna=False)

NaN                17243
diesel              2727
petrol-electric     1347
petrol               399
electric              68
Name: fuel_type, dtype: int64

In [46]:
df_f.to_csv("extracted_fuel_type.csv")

In [52]:
df_f = pd.read_csv("extracted_fuel_type.csv")

In [53]:
df_f.fuel_type =df_f.fuel_type.fillna(df_f.extracted_fuel_type_standardized)

In [54]:
df_f.fuel_type.value_counts(dropna=False)

petrol             14547
NaN                 3057
diesel              2734
petrol-electric     1378
electric              68
Name: fuel_type, dtype: int64

In [55]:
df_f

,Unnamed: 0,listing_id,fuel_type,CarCode,extracted_fuel_type,extracted_fuel_type_standardized
0,0,1030324,petrol,11382,Petrol,petrol
1,1,1021510,diesel,MISSING,MISSING CAR CODE,NaN
2,2,1026909,petrol,11386,Petrol,petrol
3,3,1019371,petrol,11846,Petrol,petrol
4,4,1031014,petrol,11875,Petrol,petrol
...,...,...,...,...,...,...
21779,21779,1010230,petrol,11998,Petrol,petrol
21780,21780,1001823,petrol,10275,Petrol,petrol
21781,21781,1027276,petrol,11125,Petrol,petrol
21782,21782,1019321,petrol,11643,Petrol,petrol


In [56]:
df_f[["listing_id", "fuel_type"]].set_index('listing_id').to_json('listing_id_TO_fuel_type.json')

ValueError: DataFrame index must be unique for orient='columns'.

In [84]:
df_f.drop(columns=["Unnamed: 0"]).drop_duplicates(subset=['listing_id'], keep='last')[["listing_id", "fuel_type"]].set_index('listing_id').to_json('listing_id_TO_fuel_type.json')

In [81]:
df_f[df_f.listing_id == 1024788]

,Unnamed: 0,listing_id,fuel_type,CarCode,extracted_fuel_type,extracted_fuel_type_standardized
16294,16294,1024788,petrol,11806,Petrol,petrol
17389,17389,1024788,petrol-electric,11806,Petrol,petrol
